# 2020 Önálló kutatási feladat

In [2]:
import pandas as pd

In [21]:
stock = pd.read_csv('..\Adatok\stock-data\SP500.csv')

stock = stock.drop(columns=['High','Low','Adj Close','Volume'])

def Change(row):
    return (row['Close']-row['Open'])
stock['Change'] = stock.apply (lambda row: Change(row), axis=1)

def label(row, threshold): 
    if abs(row['Change']) > threshold:
        if row['Change'] > 0: return int(1)
        else: return int(0)


def generate_data(sensitivity):
    stock['Label'] = stock.apply (lambda row: label(row, sensitivity), axis=1)
    stock_ready = stock.drop(columns=['Open', 'Close', 'Change'])
    return stock_ready 
    
stock


,Date,Open,Close,Change
0,2000-01-03,1469.250000,1455.219971,-14.030029
1,2000-01-04,1455.219971,1399.420044,-55.799927
2,2000-01-05,1399.420044,1402.109985,2.689941
3,2000-01-06,1402.109985,1403.449951,1.339966
4,2000-01-07,1403.449951,1441.469971,38.020020
...,...,...,...,...
5164,2020-07-14,3141.110107,3197.520020,56.409913
5165,2020-07-15,3225.979980,3226.560059,0.580079
5166,2020-07-16,3208.360107,3215.570068,7.209961
5167,2020-07-17,3224.209961,3224.729980,0.520019


### Itt tudjuk a *generate_data* függvény paraméterével állítani, hogy milyen mértékű változást veszünk figyelembe.

In [22]:
generate_data(0)

,Date,Label
0,2000-01-03,0.0
1,2000-01-04,0.0
2,2000-01-05,1.0
3,2000-01-06,1.0
4,2000-01-07,1.0
...,...,...
5164,2020-07-14,1.0
5165,2020-07-15,1.0
5166,2020-07-16,1.0
5167,2020-07-17,1.0
